In [2]:
# Importer les bibliothèques nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from keras.models import load_model
import os

In [4]:
file_path = "df_with_clusters.csv"  
df = pd.read_csv(file_path)
df

,user_number,datetime,Consommation,cluster
0,1,2016-03-01 01:19:43,0,0
1,1,2016-03-01 02:19:43,0,0
2,1,2016-03-01 03:19:43,0,0
3,1,2016-03-01 04:19:43,0,0
4,1,2016-03-01 05:39:54,0,0
...,...,...,...,...
673922,92,2017-02-28 18:08:01,0,2
673923,92,2017-02-28 19:08:06,0,2
673924,92,2017-02-28 20:08:06,27,2
673925,92,2017-02-28 21:08:06,43,2


In [5]:
# Séparer les utilisateurs par cluster
clusters = df['cluster'].unique()

# Créer un répertoire pour enregistrer les modèles
os.makedirs("autoencoder_models", exist_ok=True)

# Fonction pour entraîner un autoencoder pour chaque cluster et sauvegarder le modèle
def train_and_save_autoencoder(data, cluster_id):
    print(f"Entraînement de l'autoencoder pour le cluster {cluster_id}")
    
    # Extraire les données de consommation pour les utilisateurs dans ce cluster
    cluster_data = data[data['cluster'] == cluster_id]
    
    # Sélectionner les colonnes pertinentes pour l'entraînement
    cluster_data = cluster_data[['Consommation']]  # Vous pouvez ajouter d'autres colonnes si nécessaire
    
    # Normaliser les données de consommation
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(cluster_data)
    
    # Diviser les données en ensemble d'entraînement et de validation
    X_train, X_val = train_test_split(data_scaled, test_size=0.2, random_state=42)

    # Définir l'architecture de l'autoencoder
    input_dim = X_train.shape[1]  # Nombre de caractéristiques d'entrée (une seule pour 'Consommation')
    encoding_dim = 8  # Nombre de neurones dans la couche cachée (dimension réduite)

    # Définir les couches de l'autoencoder
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)

    # Construire le modèle autoencoder
    autoencoder = Model(inputs=input_layer, outputs=decoded)

    # Compiler le modèle
    autoencoder.compile(optimizer='adam', loss='mse')

    # Entraîner le modèle
    autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_val, X_val), shuffle=True)

    # Sauvegarder le modèle
    model_filename = f"autoencoder_models/autoencoder_cluster_{cluster_id}.h5"
    autoencoder.save(model_filename)
    print(f"Modèle du cluster {cluster_id} sauvegardé sous {model_filename}")
    
# Boucle à travers les clusters pour entraîner un autoencoder pour chaque cluster et sauvegarder les modèles
for cluster_id in clusters:
    train_and_save_autoencoder(df, cluster_id)


Entraînement de l'autoencoder pour le cluster 0
Epoch 1/50
7267/7267 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 3.6926e-05 - val_loss: 7.4323e-07
Epoch 2/50
7267/7267 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 1.7614e-06 - val_loss: 7.2063e-07
Epoch 3/50
7267/7267 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 4.4259e-06 - val_loss: 7.3571e-07
Epoch 4/50
7267/7267 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 4.8858e-06 - val_loss: 7.0733e-07
Epoch 5/50
7267/7267 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 2.0623e-06 - val_loss: 5.6905e-07
Epoch 6/50
7267/7267 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 2.1599e-06 - val_loss: 5.4448e-07
Epoch 7/50
7267/7267 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 2.8923e-06 - val_loss: 4.7651e-07
Epoch 8/50
7267/7267 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 2.4781e-06 - val_loss: 4.3758e-07
Epoch 9/50
7267/7267 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 2.3760e-06 - val_loss: 4.5562e-07
Epoch 10/50
7267/7267 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 1.9615e-06 - val_los

Modèle du cluster 0 sauvegardé sous autoencoder_models/autoencoder_cluster_0.h5
Entraînement de l'autoencoder pour le cluster 1
Epoch 1/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0916 - val_loss: 7.8041e-04
Epoch 2/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 4.8901e-04 - val_loss: 9.6044e-05
Epoch 3/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 9.5811e-05 - val_loss: 1.9708e-05
Epoch 4/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1.6841e-05 - val_loss: 6.2698e-06
Epoch 5/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 4.2021e-05 - val_loss: 4.0972e-06
Epoch 6/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 2.2457e-05 - val_loss: 3.9179e-06
Epoch 7/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 2.6013e-05 - val_loss: 3.9134e-06
Epoch 8/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 6.8738e-06 - val_loss: 3.9105e-06
Epoch 9/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 2.1339e-05 - val_loss: 3.9061e-06
Epoch 10/50


Modèle du cluster 1 sauvegardé sous autoencoder_models/autoencoder_cluster_1.h5
Entraînement de l'autoencoder pour le cluster 2
Epoch 1/50
7214/7214 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 3.3670e-06 - val_loss: 8.0684e-06
Epoch 2/50
7214/7214 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 5.1648e-06 - val_loss: 7.2325e-06
Epoch 3/50
7214/7214 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 1.0286e-05 - val_loss: 6.2581e-06
Epoch 4/50
7214/7214 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 3.1954e-06 - val_loss: 5.6655e-06
Epoch 5/50
7214/7214 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 3.6809e-06 - val_loss: 4.9813e-06
Epoch 6/50
7214/7214 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 7.0539e-06 - val_loss: 4.4928e-06
Epoch 7/50
7214/7214 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 2.7668e-06 - val_loss: 3.7079e-06
Epoch 8/50
7214/7214 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 1.8955e-06 - val_loss: 3.1994e-06
Epoch 9/50
7214/7214 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 1.9911e-06 - val_loss: 2.7042e-06

Modèle du cluster 2 sauvegardé sous autoencoder_models/autoencoder_cluster_2.h5
